# Análise de Crédito Agregado – BACEN/SCR

![Banco Central do Brasil](https://www.remessaonline.com.br/blog/wp-content/uploads/2022/10/Como-consultar-o-SCR-do-Banco-Central-1024x683.jpg.optimal.jpg)

## Introdução

O Banco Central do Brasil divulga mensalmente um conjunto abrangente de informações agregadas sobre as operações de crédito realizadas no país, com base nos dados do Sistema de Informações de Crédito (SCR).

Este projeto tem como objetivo explorar, analisar e visualizar os dados públicos do SCR, que abrangem:

- Carteira Ativa
- Inadimplência
- Ativos Problemáticos

Os dados estão disponíveis com recortes por tipo de cliente (Pessoa Física ou Jurídica), modalidade de crédito, região geográfica, setor econômico (CNAE), porte/rendimento dos clientes, entre outros. A série histórica cobre o período de **junho de 2012 até os dias atuais**, com atualizações mensais.

> As informações aqui tratadas são extraídas dos dados abertos publicados pelo Banco Central, podendo haver divergências em relação a outras publicações devido a diferenças nas fontes e metodologias de agregação.

---



# Importação Bibliotecas

In [42]:
# Imports para análise e manipulação de dados
import pandas as pd
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Ferramentas para pré-processamento e análise estatística
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Configurações de estilo para gráficos
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Exibir todas as colunas no head()
pd.set_option('display.max_columns', None)



# Visualização dos Dados

In [43]:
df = pd.read_csv('../data/planilhas_ano_2024/planilha_202412.csv', sep=';')

In [44]:
df.head(2)

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,origem,indexador,numero_de_operacoes,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
0,2024-12-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,Sem destinação específica,Prefixado,609,"4225505,49","1245559,17","103358,78","17925,05","11025,00","0,00","186098,00","5789471,49","150711,24","276455,95"
1,2024-12-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,Sem destinação específica,Prefixado,615,"1770659,61","4592675,73","10504470,83","7770023,25","7407915,67","0,00","186870,01","32232615,10","1314588,78","1314588,78"


# Informações dos dados

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972320 entries, 0 to 972319
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   data_base                       972320 non-null  object
 1   uf                              972320 non-null  object
 2   tcb                             972320 non-null  object
 3   sr                              970196 non-null  object
 4   cliente                         972320 non-null  object
 5   ocupacao                        972320 non-null  object
 6   cnae_secao                      972320 non-null  object
 7   cnae_subclasse                  972320 non-null  object
 8   porte                           972320 non-null  object
 9   modalidade                      972320 non-null  object
 10  origem                          972320 non-null  object
 11  indexador                       972320 non-null  object
 12  numero_de_operacoes           

In [46]:
df.isnull().sum()
# Verificando a presença de valores nulos

data_base                            0
uf                                   0
tcb                                  0
sr                                2124
cliente                              0
ocupacao                             0
cnae_secao                           0
cnae_subclasse                       0
porte                                0
modalidade                           0
origem                               0
indexador                            0
numero_de_operacoes                  0
a_vencer_ate_90_dias                 0
a_vencer_de_91_ate_360_dias          0
a_vencer_de_361_ate_1080_dias        0
a_vencer_de_1081_ate_1800_dias       0
a_vencer_de_1801_ate_5400_dias       0
a_vencer_acima_de_5400_dias          0
vencido_acima_de_15_dias             0
carteira_ativa                       0
carteira_inadimplida_arrastada       0
ativo_problematico                   0
dtype: int64

# Dicionário de Dados – Base SCR (Banco Central)

A base contém dados agregados sobre operações de crédito, divulgados mensalmente pelo Banco Central do Brasil, com segmentações por tipo de cliente, modalidade, região, setor econômico e prazo de vencimento das operações.

| Nome da Coluna                          | Tipo     | Descrição |
|----------------------------------------|----------|-----------|
| `data_base`                             | data     | Data de referência da informação (último dia do mês) |
| `uf`                                    | string   | Unidade da Federação (ex: AC, SP, RJ) |
| `tcb`                                   | string   | Tipo de Conglomerado Bancário (ex: Bancário, Cooperativo, etc.) |
| `sr`                                    | string   | Superintendência Regional ou agrupamento geográfico |
| `cliente`                               | string   | Tipo de cliente: Pessoa Física (PF) ou Jurídica (PJ) |
| `ocupacao`                              | string   | Ocupação do cliente PF (ex: Aposentado/pensionista, Trabalhador formal etc.) |
| `cnae_secao`                            | string   | Seção da CNAE (atividade econômica principal – PJ) |
| `cnae_subclasse`                        | string   | Subclasse da CNAE (atividade econômica mais detalhada – PJ) |
| `porte`                                 | string   | Porte da empresa (PJ) ou faixa de renda (PF) |
| `modalidade`                            | string   | Modalidade da operação de crédito (ex: Cartão de crédito, Consignado etc.) |
| `a_vencer_ate_90_dias`                  | float    | Valor das operações com vencimento em até 90 dias |
| `a_vencer_de_91_ate_360_dias`           | float    | Valor das operações com vencimento entre 91 e 360 dias |
| `a_vencer_de_361_ate_1080_dias`         | float    | Valor das operações com vencimento entre 361 e 1080 dias |
| `a_vencer_de_1081_ate_1800_dias`        | float    | Valor das operações com vencimento entre 1081 e 1800 dias |
| `a_vencer_de_1801_ate_5400_dias`        | float    | Valor das operações com vencimento entre 1801 e 5400 dias |
| `a_vencer_acima_de_5400_dias`           | float    | Valor das operações com vencimento acima de 5400 dias |
| `vencido_acima_de_15_dias`              | float    | Valor total vencido há mais de 15 dias |
| `carteira_ativa`                        | float    | Total da carteira ativa (em dia e vencida) |
| `carteira_inadimplida_arrastada`        | float    | Valor de operações com inadimplência acumulada |
| `ativo_problematico`                    | float    | Soma de inadimplência e reestruturações (ativos problemáticos) |


# Variáveis Categóricas – Segmentação e Perfil de Cliente

| Variável              | Uso em Credit Scoring                                                   |
|-----------------------|-------------------------------------------------------------------------|
| `cliente`             | Segmenta PF ou PJ (modelos separados ou variáveis distintas)            |
| `ocupacao`            | Relevante para PF – Ocupações estáveis tendem a menor inadimplência     |
| `cnae_secao`          | Para PJ – tipo de atividade influencia risco de crédito                 |
| `cnae_subclasse`      | Detalhamento da atividade econômica da PJ                               |
| `porte`               | PJ: porte da empresa; PF: faixa de renda                                |
| `modalidade`          | Cada tipo de produto possui perfil de risco diferente                   |
| `uf`, `sr`            | Risco regional – inadimplência varia por localização                    |

---

# Variáveis Quantitativas – Valores de Crédito e Inadimplência

| Variável                          | Uso em Credit Scoring                                           |
|-----------------------------------|------------------------------------------------------------------|
| `a_vencer_ate_90_dias` até `a_vencer_acima_de_5400_dias` | Perfil de prazo da dívida – risco varia com o tempo de vencimento |
| `vencido_acima_de_15_dias`       | Indica inadimplência em estágio inicial                         |
| `carteira_ativa`                 | Total de operações de crédito registradas                       |
| `carteira_inadimplida_arrastada` | Valor vencido de longo prazo (inadimplência grave)              |
| `ativo_problematico`             | Soma da inadimplência com reestruturações de crédito            |



# Variável Target: `carteira_inadimplida_arrastada`

A variável `carteira_inadimplida_arrastada` representa o valor total das operações de crédito que estão inadimplentes **há mais de 90 dias**. Essas operações são consideradas **créditos de alto risco**, pois os atrasos prolongados indicam forte probabilidade de **calote ou perda financeira** para a instituição financeira.

---

## Importância em Modelagem de Risco de Crédito

Esta variável é amplamente utilizada como **variável target (resposta)** em modelos de **Credit Scoring**, pois:

- Captura inadimplência de forma consolidada (não apenas atraso pontual);
- É adequada para prever o risco de crédito com base em características do cliente e do crédito;
- É compatível com abordagens de classificação e regressão.


# Conversão da variável data_base para o formato time

In [47]:
df["data_base"] = pd.to_datetime(df["data_base"], format="%Y-%m-%d")

# Conversão do tipo das variáveis (object to float)

In [48]:
colunas_numericas = [
    "a_vencer_ate_90_dias",
    "a_vencer_de_91_ate_360_dias",
    "a_vencer_de_361_ate_1080_dias",
    "a_vencer_de_1081_ate_1800_dias",
    "a_vencer_de_1801_ate_5400_dias",
    "a_vencer_acima_de_5400_dias",
    "vencido_acima_de_15_dias",
    "carteira_ativa",
    "carteira_inadimplida_arrastada",
    "ativo_problematico"
]

for col in colunas_numericas:
    df[col] = df[col].astype(str) 
    df[col] = df[col].str.replace(".", "", regex=False)
    df[col] = df[col].str.replace(",", ".", regex=False)
    df[col] = df[col].astype(float)


# Classificação binária (inadimplente vs. adimplente)

Podemos criar uma variável auxiliar que identifica se houve inadimplência:

In [49]:
df["inadimplente"] = (df["carteira_inadimplida_arrastada"] > 0).astype(int)


In [50]:
df.head(2)

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,origem,indexador,numero_de_operacoes,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico,inadimplente
0,2024-12-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,Sem destinação específica,Prefixado,609,4225505.49,1245559.17,103358.78,17925.05,11025.00,0.0,186098.00,5789471.49,150711.24,276455.95,1
1,2024-12-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,Sem destinação específica,Prefixado,615,1770659.61,4592675.73,10504470.83,7770023.25,7407915.67,0.0,186870.01,32232615.10,1314588.78,1314588.78,1
